In [0]:
from pyspark.sql.types import FloatType
from pyspark.sql.functions import max,count,col,desc,rank
from pyspark.sql import Window

In [0]:
df_of_2021 = spark.read.option("header", "true").csv("dbfs:/FileStore/big-data-processing/world_happiness_report_2021.csv")
other_df = spark.read.option("header", "true").csv("dbfs:/FileStore/big-data-processing/world_happiness_report.csv")

In [0]:
df_of_2021.printSchema()

root
 |-- Country name: string (nullable = true)
 |-- Regional indicator: string (nullable = true)
 |-- Ladder score: string (nullable = true)
 |-- Standard error of ladder score: string (nullable = true)
 |-- upperwhisker: string (nullable = true)
 |-- lowerwhisker: string (nullable = true)
 |-- Logged GDP per capita: string (nullable = true)
 |-- Social support: string (nullable = true)
 |-- Healthy life expectancy: string (nullable = true)
 |-- Freedom to make life choices: string (nullable = true)
 |-- Generosity: string (nullable = true)
 |-- Perceptions of corruption: string (nullable = true)
 |-- Ladder score in Dystopia: string (nullable = true)
 |-- Explained by: Log GDP per capita: string (nullable = true)
 |-- Explained by: Social support: string (nullable = true)
 |-- Explained by: Healthy life expectancy: string (nullable = true)
 |-- Explained by: Freedom to make life choices: string (nullable = true)
 |-- Explained by: Generosity: string (nullable = true)
 |-- Explained 

In [0]:
# ¿Cual es el pais mas feliz del 2021 segun la data? Finland

df_of_2021.select('Country Name','Ladder Score').show(1)

+------------+------------+
|Country Name|Ladder Score|
+------------+------------+
|     Finland|       7.842|
+------------+------------+
only showing top 1 row



In [0]:
# ¿Cual es el pais mas feliz del 2021 por continente segun la data?

df = df_of_2021.withColumn("Ladder Score", df_of_2021['Ladder Score'].cast('float'))

max_ladder_score_per_region = df.groupby(['Regional Indicator']).agg(max('Ladder Score').alias('Ladder Score'))
max_ladder_score_with_countries = max_ladder_score_per_region.join(df, ['Regional Indicator','Ladder Score'],'inner')
max_ladder_score_with_countries.select('Regional Indicator', 'Ladder Score', 'Country Name').show(truncate=False)



+----------------------------------+------------+------------------------+
|Regional Indicator                |Ladder Score|Country Name            |
+----------------------------------+------------+------------------------+
|Western Europe                    |7.842       |Finland                 |
|North America and ANZ             |7.277       |New Zealand             |
|Middle East and North Africa      |7.157       |Israel                  |
|Latin America and Caribbean       |7.069       |Costa Rica              |
|Central and Eastern Europe        |6.965       |Czech Republic          |
|East Asia                         |6.584       |Taiwan Province of China|
|Southeast Asia                    |6.377       |Singapore               |
|Commonwealth of Independent States|6.179       |Uzbekistan              |
|Sub-Saharan Africa                |6.049       |Mauritius               |
|South Asia                        |5.269       |Nepal                   |
+------------------------

In [0]:
# ¿Cual es el pais que mas veces ocupó el primer lugar en todos los años? Denmark


df_other = other_df.withColumn("Life Ladder", other_df['Life Ladder'].cast('float'))
max_scores_per_year = df_other.groupby('year').agg(max('Life Ladder').alias('Life Ladder'))
max_scores_countries = max_scores_per_year.join(df_other,['year','Life Ladder'], 'inner')
times_on_top = max_scores_countries.groupby('Country name').agg(count('Country name').alias('Times on Top'))
times_on_top.orderBy(desc('Times on Top')).show()


+------------+------------+
|Country name|Times on Top|
+------------+------------+
|     Denmark|           7|
|     Finland|           6|
|      Norway|           1|
| Switzerland|           1|
|      Canada|           1|
+------------+------------+



In [0]:
# ¿Que puesto de felicidad tiene el pais con mayor GDP del 2020? 13-Ireland

countries_2020 = other_df.withColumn("Log GDP per capita", other_df['Log GDP per capita'].cast('float')).filter(other_df.year == 2020)
max_country_gdp = countries_2020.select('Country name','Log GDP per capita').orderBy(desc('Log GDP per capita')).limit(1)
top = Window.orderBy(desc('Life Ladder'))
countries_ranking =  countries_2020.withColumn('rank',rank().over(top)).select('Country name','rank','Log GDP per capita')
position = max_country_gdp.join(countries_ranking, ['Country name','Log GDP per capita'], 'inner')
position.show()

+------------+------------------+----+
|Country name|Log GDP per capita|rank|
+------------+------------------+----+
|     Ireland|            11.323|  13|
+------------+------------------+----+



In [0]:
# ¿Cual es el pais con mayor expectativa de vida y cuanto tenia en ese indicador en el 2019?

countries_2019 = other_df.withColumn("Log GDP per capita", other_df['Log GDP per capita'].cast('float')).filter(other_df.year == 2019)
countries_2019.select('Country name','Healthy life expectancy at birth').orderBy(desc('Healthy life expectancy at birth')).show(1)

+------------+--------------------------------+
|Country name|Healthy life expectancy at birth|
+------------+--------------------------------+
|   Singapore|                          77.100|
+------------+--------------------------------+
only showing top 1 row



In [0]:
other_df.printSchema()

root
 |-- Country name: string (nullable = true)
 |-- year: string (nullable = true)
 |-- Life Ladder: string (nullable = true)
 |-- Log GDP per capita: string (nullable = true)
 |-- Social support: string (nullable = true)
 |-- Healthy life expectancy at birth: string (nullable = true)
 |-- Freedom to make life choices: string (nullable = true)
 |-- Generosity: string (nullable = true)
 |-- Perceptions of corruption: string (nullable = true)
 |-- Positive affect: string (nullable = true)
 |-- Negative affect: string (nullable = true)

